#### 📝 Step 3.1: Convert shape_ann/ to YOLO Format



In [10]:
import os

# 🔹 Define Paths
dataset_path = r"C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2"
shape_ann_path = os.path.join(dataset_path, "shape_ann")  # Annotation folder
labels_shape_path = os.path.join(dataset_path, "labels_shape")  # YOLO output folder
os.makedirs(labels_shape_path, exist_ok=True)

def convert_bbox_to_yolo(x, y, w, h, img_w, img_h):
    """Convert absolute bbox coordinates to YOLO format (normalized)."""
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w = w / img_w
    h = h / img_h
    return x_center, y_center, w, h

# 🔹 Process Each Annotation File
for annotation_file in os.listdir(shape_ann_path):
    if not annotation_file.endswith(".txt"):
        continue

    input_path = os.path.join(shape_ann_path, annotation_file)
    output_path = os.path.join(labels_shape_path, annotation_file)

    with open(input_path, 'r') as file, open(output_path, 'w') as yolo_file:
        lines = file.readlines()

        for line in lines:
            data = line.strip().split()

            if len(data) < 5:  # Ensure enough data
                continue  

            # 🔹 Skip the filename if it contains an image name
            if data[0].endswith(('.png', '.jpg')):  
                data = data[1:]  

            try:
                class_id = int(data[0])  # Class ID
                x, y, w, h = map(float, data[1:5])  # Bbox values
                img_w, img_h = 256, 256  # Modify based on actual image size

                # 🔹 Convert to YOLO format
                x_center, y_center, w_norm, h_norm = convert_bbox_to_yolo(x, y, w, h, img_w, img_h)

                # 🔹 Save YOLO annotation
                yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

            except ValueError:
                print(f"Skipping invalid line in {annotation_file}: {line}")

print("✅ Shape annotations converted! YOLO labels saved in 'labels_shape/' folder.")


✅ Shape annotations converted! YOLO labels saved in 'labels_shape/' folder.


#### 📝 Step 3.2: Convert texture_ann/ to YOLO Format


In [11]:
# Paths
texture_ann_path = os.path.join(dataset_path, "texture_ann")
labels_texture_path = os.path.join(dataset_path, "labels_texture")
os.makedirs(labels_texture_path, exist_ok=True)

# Loop through train, val, and test
for subset in ["train", "val", "test"]:
    ann_subset_path = os.path.join(texture_ann_path, subset)
    labels_subset_path = os.path.join(labels_texture_path, subset)
    os.makedirs(labels_subset_path, exist_ok=True)

    for ann_file in os.listdir(ann_subset_path):
        if ann_file.endswith(".txt"):
            ann_path = os.path.join(ann_subset_path, ann_file)
            label_file = os.path.join(labels_subset_path, ann_file)

            with open(ann_path, "r") as f, open(label_file, "w") as yolo_f:
                for line in f.readlines():
                    data = line.strip().split()
                    if len(data) < 5:
                        continue  # Skip invalid lines
                    
                    # Read bounding box values
                    class_id = int(data[0])  # Class ID
                    x, y, w, h = map(float, data[1:5])  # Bbox values
                    img_w, img_h = 256, 256  # Modify if needed

                    # Convert to YOLO format
                    x_center, y_center, w_norm, h_norm = convert_bbox_to_yolo(x, y, w, h, img_w, img_h)

                    # Save YOLO annotation
                    yolo_f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

print("✅ Texture annotations converted! YOLO labels saved in 'labels_texture/' folder.")


✅ Texture annotations converted! YOLO labels saved in 'labels_texture/' folder.


In [3]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO("yolov8s.pt")  # You can also use 'yolov8m.pt' or 'yolov8l.pt'

# Train the model on DeepFashion2 dataset
model.train(data=r"C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\deepfashion2.yaml", epochs=10, imgsz=320)


Ultralytics 8.3.97  Python-3.12.9 torch-2.2.1+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\deepfashion2.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\train_images.cache... 0 images, 10335 backgrounds, 0 corrupt: 100%|██████████| 10335/10335 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\train_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\test_images.cache... 0 images, 1149 backgrounds, 0 corrupt: 100%|██████████| 1149/1149 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\test_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to c:\Users\raiha\OneDrive\Desktop\AR-Shopping-Assistant\runs\detect\train8\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000455, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


TensorBoard: model graph visualization added 
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to c:\Users\raiha\OneDrive\Desktop\AR-Shopping-Assistant\runs\detect\train8
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G          0      19.55          0          0        320:  85%|████████▍ | 549/646 [29:08<05:08,  3.19s/it]


KeyboardInterrupt: 

In [ ]:
# Validate the model
model.val(data=r"C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\deepfashion2.yaml")


In [ ]:
# Run inference on test images
results = model("C:/Users/raiha/OneDrive/Desktop/cv_dataset/deepfashion2/test_images/")
